In [14]:
import pandas as pd
from ProjectTools import OpenBeds
from ProjectTools.OpenBeds import filterDepth

In [24]:
dry = True

if dry == True:
    top_hmc_genes = pd.read_excel('./top_hmc_genes_testSet.xlsx').to_dict()
    top_hmc_genes.pop("Unnamed: 0")

    tab_bed_path = './test_data/CRR008807_TAB_merged.bedGraph.gz.bismark.zero.cov_sub.bed'
    nano_3mod_path = './test_data/prom_R10.4.1_E8.2_WGS_brain_0.9.1_mods_sub.bed'
    nano_mc_df, nano_hmc_df = map(filterDepth, OpenBeds.get_nanopore_threeMod_wStrand(nano_3mod_path))
    tab_df = filterDepth(OpenBeds.get_bismark(tab_bed_path, "5hmC"))

else:
    top_hmc_genes = pd.read_excel('./top_hmc_genes.xlsx').to_dict()
    top_hmc_genes.pop("Unnamed: 0")

    tab_bed_path = './data/TAB_data/CRR008807_TAB_merged.bedGraph.gz.bismark.zero.cov'
    nano_3mod_path = './data/prom_R10.4.1_E8.2_WGS_brain_0.9.1_mods.bed'
    nano_mc_df, nano_hmc_df = map(filterDepth, OpenBeds.get_nanopore_threeMod_wStrand(nano_3mod_path))
    tab_df = filterDepth(OpenBeds.get_bismark(tab_bed_path, "5hmC"))

In [41]:
from pybedtools import BedTool

nano_hmc_df = nano_hmc_df[["chromosome", "chromStart", "chromEnd", "readCount", "percentMeth", "strand"]]

nano_intersect = BedTool.intersect(
    BedTool.from_dataframe(nano_hmc_df),
    BedTool('./feature_references/fig5_features/mm39_RefSeqC_select_merged_modified.bed'),
    wb=True, s=True).to_dataframe(names=["chromosome", "chromStart", "chromEnd", "readCount", "percentMeth", "strand", ".1", ".2", ".3", "geneName", ".4", ".5"]).drop(
    columns=["readCount", ".1", ".2", ".3", ".4", ".5"]
    )

In [42]:
nano_intersect

,chromosome,chromStart,chromEnd,percentMeth,strand,geneName
0,chr1,3284715,3284716,10.00,-,Xkr4
1,chr1,3284757,3284758,8.33,-,Xkr4
2,chr1,3285115,3285116,16.67,-,Xkr4
3,chr1,3286231,3286232,8.33,-,Xkr4
4,chr1,3286357,3286358,8.33,-,Xkr4
...,...,...,...,...,...,...
220195,chr1,72576275,72576276,0.00,-,Marchf4
220196,chr1,72576279,72576280,0.00,-,Marchf4
220197,chr1,72576283,72576284,0.00,-,Marchf4
220198,chr1,72576291,72576292,0.00,-,Marchf4


In [10]:
pd.DataFrame(top_hmc_genes)

,gene_name,ratio_TAB,CpG_count_TAB,log2_TAB,ratio_Nano,CpG_count_Nano,log2_Nano,log2_difference
0,Gm15880,0.678091,57,-0.560450,0.509106,112,-0.973962,0.413512
1,Pan3,0.678606,453,-0.559353,0.507867,855,-0.977479,0.418125
2,Fbxw11,0.703372,341,-0.507640,0.518866,826,-0.946565,0.438926
3,Tspan5,0.687292,799,-0.541005,0.468553,2213,-1.093717,0.552711
4,Nedd4l,0.696387,1554,-0.522039,0.474245,3778,-1.076296,0.554257
5,Jmjd1c,0.725641,425,-0.462673,0.489034,989,-1.031992,0.569320
6,Ppp3ca,0.814528,1140,-0.295964,0.536714,2297,-0.897776,0.601811
7,Tcf4,0.737417,1944,-0.439448,0.479584,4032,-1.060145,0.620697
8,Dyrk1a,0.742249,507,-0.430025,0.480054,1095,-1.058730,0.628705
9,Lhfp,0.727954,843,-0.458081,0.465749,2025,-1.102374,0.644293


In [11]:
import subprocess

genes_info = {}

for gene_name in top_hmc_genes["gene_name"].values():
    gene_info = subprocess.check_output(
        "grep {gene_name} ./feature_references/fig5_features/mm39_RefSeqC_select_merged_modified.bed".format(
        gene_name=gene_name), shell=True).decode(
        "UTF-8").split()
    gene_entry = {
        gene_name : 
        {
        "chromosome" : gene_info[0],
        "chromStart" : gene_info[1],
        "chromEnd" : gene_info[2],
        "strand" : gene_info[5]
        }
        }
    
    genes_info.update(gene_entry)

In [12]:
pd.DataFrame(genes_info).T

,chromosome,chromStart,chromEnd,strand
Gm15880,chr7,80285764,80294132,+
Pan3,chr5,147366965,147485311,+
Fbxw11,chr11,32592554,32696814,+
Tspan5,chr3,138447968,138610194,+
Nedd4l,chr18,65020826,65350897,+
Jmjd1c,chr10,66962866,67092105,+
Ppp3ca,chr3,136375826,136643488,+
Tcf4,chr18,69477562,69821038,+
Dyrk1a,chr16,94371064,94496378,+
Lhfp,chr13,94194303,94331917,+


In [29]:
target_bed = pd.DataFrame(genes_info).T.reset_index()[["chromosome", "chromStart", "chromEnd"]]
target_bed.to_csv('./feature_references/fig7_genes/target_genes.bed', sep="\t", header=None, index=None)